In [2]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [4]:
from keras.models import model_from_json
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)


In [5]:
emotion_model.load_weights("model/emotion_model.h5")

print("Loaded model from disk")

Loaded model from disk


In [6]:
from keras.models import load_model



# Charger le modèle de détection d'état des yeux
eye_model = load_model('yawn_detection.h5')
print("Loaded model from disk")

Loaded model from disk


In [26]:
import cv2
import numpy as np
from keras.models import load_model



# Définir les classes
eye_dict = { 0: "Closed Eyes" ,1: "Open Eyes"}

# Charger les classificateurs en cascade Haar pour la détection de visage et des yeux
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def preprocess_image(frame):
    # Vérifier si l'image est correctement lue
    if frame is None:
        return None

    # Vérifier le nombre de canaux de couleur dans l'image
    if len(frame.shape) == 3 and frame.shape[2] == 3:  # Vérifier si l'image est en couleur (3 canaux)
        # Convertir l'image en niveaux de gris
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        # L'image est déjà en niveaux de gris
        gray_frame = frame

    # Redimensionnez l'image à la taille attendue par votre modèle (64x64 dans ce cas)
    resized_frame = cv2.resize(gray_frame, (64, 64))

    # Normaliser l'image
    normalized_frame = resized_frame / 255.0

    # Ajouter une dimension supplémentaire pour correspondre à la forme d'entrée attendue du modèle
    preprocessed_frame = np.expand_dims(normalized_frame, axis=0)
    # Ajouter une dimension supplémentaire pour correspondre à la forme d'entrée attendue du modèle
    preprocessed_frame = np.expand_dims(preprocessed_frame, axis=-1)

    return preprocessed_frame

# Capturez le flux vidéo de la webcam (index 0)
cap = cv2.VideoCapture(0)

# Définir le nombre d'images à sauter avant de détecter à nouveau les visages et les yeux
skip_frames = 10
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % skip_frames != 0:
        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

            preprocessed_eye = preprocess_image(roi_gray[ey:ey+eh, ex:ex+ew])
            if preprocessed_eye is None:
                continue

            prediction = model.predict(preprocessed_eye)
            predicted_class_index = np.argmax(prediction)
            label = eye_dict[predicted_class_index]

            
        
        # Prétraiter l'image pour la détection d'émotion
        cropped_img = cv2.resize(roi_gray, (48, 48))
        cropped_img = np.expand_dims(np.expand_dims(cropped_img, -1), 0)
        
        # Prédire l'émotion
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        
        # Afficher l'émotion prédite sur le visage détecté
        eye_color = (36, 255, 12)  # Couleur verte
        cv2.putText(frame, label, (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, eye_color, 2)

# Pour l'émotion détectée
        emotion_color = (255, 0, 0)  # Couleur rouge
        cv2.putText(frame, emotion_dict[maxindex], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, emotion_color, 2, cv2.LINE_AA)


    # Afficher la frame
    cv2.imshow('Emotion and Eye State Detection', frame)
    
    # Quitter la boucle si la touche 'q' est enfoncée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 45ms/step
